In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

2025-04-19 18:04:36.319236: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/sumitsontakke/Documents/Dev/PES/mlenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Check for GPU availability
if tf.config.list_physical_devices('GPU'):
    print("GPU is available and in use!")
else:
    print("No GPU is available. Make sure the runtime is set to GPU.")

# Optionally, you can print the GPU details
print("TensorFlow Devices:")
print(tf.config.list_physical_devices())

No GPU is available. Make sure the runtime is set to GPU.
TensorFlow Devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)


X_train shape: (50000, 32, 32, 3) y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3) y_test shape: (10000, 1)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV


def build_model1(learning_rate=0.001, dropout1=0.3, dropout2=0.2):
    model = Sequential([
        Flatten(input_shape=(32, 32, 3)),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(dropout1),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(dropout2),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

param_grid = {
    "model__learning_rate": [0.001, 0.0005],
    "model__dropout1": [0.3],
    "model__dropout2": [0.2],
    "batch_size": [64, 32],
    "epochs": [50],
}

model = KerasClassifier(model=build_model1, verbose=1, validation_split=0.2)

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2, n_jobs=1)
# RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=param_grid,
                                   n_iter=4,  # You can increase this if it's running fine
                                   cv=2,
                                   verbose=2,
                                   n_jobs=1)

# Start training
random_search.fit(X_train, y_train)

# Print best result
print("Best parameters found: ", random_search.best_params_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


/Users/sumitsontakke/Documents/Dev/PES/mlenv/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 43ms/step - accuracy: 0.2685 - loss: 2.2206 - val_accuracy: 0.3270 - val_loss: 1.9400
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.3577 - loss: 1.8285 - val_accuracy: 0.3378 - val_loss: 1.8751
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.3831 - loss: 1.7369 - val_accuracy: 0.3422 - val_loss: 1.8460
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.4039 - loss: 1.6868 - val_accuracy: 0.3332 - val_loss: 1.9454
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.4122 - loss: 1.6618 - val_accuracy: 0.3804 - val_loss: 1.7566
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.4123 - loss: 1.6595 - val_accuracy: 0.3988 - val_loss: 1.7071
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.4241 - loss: 1.6146 - val_accuracy: 0.4198 - val_loss: 1.6453
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.4370 - loss: 1.5818 - 

KeyboardInterrupt: 